# Reorganized Analysis Workflow

## 1. Load the required data and models from Google Drive

**Reasoning**:
Load the necessary data (tokenized texts and dates), the trained LDA model, and the corpus from Google Drive.

In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
from gensim.models.ldamodel import LdaModel
from gensim import corpora
import os

# Define paths to the saved files
model_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/lda_30_topics/lda_model_30_topics.gensim'
corpus_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm'
excel_path = '/content/drive/MyDrive/Colab Notebooks/alex/tokenized_newspaper_data.xlsx'

# Load the LDA model
loaded_lda_model = None
try:
    loaded_lda_model = LdaModel.load(model_path)
    print(f"LDA model loaded successfully from {model_path}")
except FileNotFoundError:
    print(f"Error: LDA model file not found at {model_path}. Cannot proceed.")
except Exception as e:
    print(f"An error occurred while loading the LDA model: {e}")

# Load the corpus
corpus = None
try:
    corpus = corpora.MmCorpus(corpus_path)
    print(f"Corpus loaded successfully from {corpus_path}")
except FileNotFoundError:
    print(f"Error: Corpus file not found at {corpus_path}. Cannot proceed.")
except Exception as e:
    print(f"An error occurred while loading the corpus: {e}")

# Load the tokenized data from Excel
tokenized_data_df = None
texts = None
try:
    tokenized_data_df = pd.read_excel(excel_path)
    print(f"Excel file loaded successfully from {excel_path}")
    if 'Main_text_tokenized' in tokenized_data_df.columns:
        texts = tokenized_data_df['Main_text_tokenized'].tolist()
        print(f"Extracted {len(texts)} documents for texts variable.")
    else:
        print("Could not find 'Main_text_tokenized' column in the Excel file. Cannot proceed.")

except FileNotFoundError:
    print(f"Error: Tokenized data Excel file not found at {excel_path}. Cannot proceed.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

# Function to find the dominant topic in each sentence (reused from previous work)
def format_topics_sentences(ldamodel, corpus, texts=None):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None and len(texts) == len(sent_topics_df):
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    elif texts is not None:
         print(f"Warning: Length of texts ({len(texts)}) does not match length of dominant topics ({len(sent_topics_df)}). Text column will not be added.")

    return(sent_topics_df)

# Create df_dominant_topic and add Year column if data is loaded
df_dominant_topic = None
if loaded_lda_model is not None and corpus is not None and texts is not None and tokenized_data_df is not None:
    try:
        df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

        if not df_topic_sents_keywords.empty:
            df_dominant_topic = df_topic_sents_keywords.reset_index()
            df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
            print("\n'df_dominant_topic' DataFrame created.")

            if 'date' in tokenized_data_df.columns and len(tokenized_data_df) == len(df_dominant_topic):
                try:
                    dates = pd.to_datetime(tokenized_data_df['date'], errors='coerce')
                    years = dates.dt.year.apply(lambda x: int(x) if pd.notna(x) else None).tolist()

                    if len(years) == len(df_dominant_topic):
                        df_dominant_topic['Year'] = years
                        print("Year column added to df_dominant_topic from 'date' column.")
                        # display(df_dominant_topic.head()) # Optional: display head after adding year
                    else:
                        print(f"Mismatch in number of documents: {len(years)} years extracted vs {len(df_dominant_topic)} in df_dominant_topic. Year column not added.")
                except Exception as e:
                    print(f"An error occurred while extracting years from 'date' column: {e}. Year column not added.")
            else:
                print("'date' column not found in tokenized_data_df or length mismatch. Cannot add Year column.")
        else:
            print("'df_topic_sents_keywords' is empty. Cannot create 'df_dominant_topic'.")
    except Exception as e:
        print(f"An error occurred while creating or processing 'df_dominant_topic': {e}.")
else:
    print("Cannot create 'df_dominant_topic' as necessary components are not available.")


# Create yearly_dataframes if df_dominant_topic was created successfully with Year
yearly_dataframes = None
if df_dominant_topic is not None and 'Year' in df_dominant_topic.columns:
    print("\nProceeding with filtering by year.")
    unique_years = df_dominant_topic['Year'].dropna().unique()
    print(f"Unique years found: {unique_years}")

    yearly_dataframes = {}
    for year in unique_years:
        yearly_df = df_dominant_topic[df_dominant_topic['Year'] == year].copy()
        yearly_dataframes[year] = yearly_df
        print(f"Filtered dataframe for year {year} with {len(yearly_df)} documents.")

    if yearly_dataframes:
        first_year = list(yearly_dataframes.keys())[0]
        print(f"\nDisplaying head of dataframe for year {first_year}:")
        display(yearly_dataframes[first_year].head())
    else:
        print("No yearly dataframes created.")
else:
    print("\nSkipping filtering by year as df_dominant_topic with Year column is not available.")


# Prepare yearly processed texts if yearly_dataframes are available
yearly_processed_texts = {}
if yearly_dataframes:
    print("\nProcessing texts for each year...")
    for year, df_year in yearly_dataframes.items():
        print(f"Processing documents for year: {year}")
        texts_for_year = df_year['Text'].tolist()
        processed_texts = []
        for text_list_str in texts_for_year:
            if isinstance(text_list_str, str):
                try:
                    text_list = eval(text_list_str)
                    if isinstance(text_list, list):
                        processed_texts.append([str(token) for token in text_list if isinstance(token, (str, int, float))])
                    else:
                        processed_texts.append([])
                except:
                    processed_texts.append([])
            else:
                processed_texts.append([])
        yearly_processed_texts[year] = processed_texts
        print(f"Processed {len(processed_texts)} documents for year {year}.")
    print("Finished processing texts for all years.")
else:
    print("Skipping processing texts as yearly_dataframes are not available.")

LDA model loaded successfully from /content/drive/MyDrive/Colab Notebooks/alex/models/lda_30_topics/lda_model_30_topics.gensim
Corpus loaded successfully from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm
Excel file loaded successfully from /content/drive/MyDrive/Colab Notebooks/alex/tokenized_newspaper_data.xlsx
Extracted 150219 documents for texts variable.

'df_dominant_topic' DataFrame created.
Year column added to df_dominant_topic from 'date' column.

Proceeding with filtering by year.
Unique years found: [2002. 2003. 2004. 2006. 2007. 2009. 2010. 2011. 2012. 2013. 2016. 2017.
 2018. 2019. 2021. 2023. 2024. 2025. 2005. 2008. 2014. 2015. 2020. 2022.]
Filtered dataframe for year 2002.0 with 9158 documents.
Filtered dataframe for year 2003.0 with 10338 documents.
Filtered dataframe for year 2004.0 with 9014 documents.
Filtered dataframe for year 2006.0 with 8764 documents.
Filtered dataframe for year 2007.0 with 7396 documents.
Filtered dataframe

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Year
0,0,22,0.3935,"said, like, would, one, get, going, time, year...","['14', 'year', 'coaching', '39', 'year', 'livi...",2002.0
1,1,22,0.2309,"said, like, would, one, get, going, time, year...","['see', 'sidebar', '02c', 'ruston', 'la', 'din...",2002.0
2,2,22,0.3757,"said, like, would, one, get, going, time, year...","['gave', 'garden', 'exactly', 'missed', 'lonel...",2002.0
3,3,22,0.3426,"said, like, would, one, get, going, time, year...","['indianapolis', 'ten', 'year', 'ago', 'chuck'...",2002.0
46,46,16,0.5945,"u, world, olympic, 2023, championship, united,...","['look', 'top', 'five', 'upperechelon', 'playe...",2002.0



Processing texts for each year...
Processing documents for year: 2002.0
Processed 9158 documents for year 2002.0.
Processing documents for year: 2003.0
Processed 10338 documents for year 2003.0.
Processing documents for year: 2004.0
Processed 9014 documents for year 2004.0.
Processing documents for year: 2006.0
Processed 8764 documents for year 2006.0.
Processing documents for year: 2007.0
Processed 7396 documents for year 2007.0.
Processing documents for year: 2009.0
Processed 6762 documents for year 2009.0.
Processing documents for year: 2010.0
Processed 6744 documents for year 2010.0.
Processing documents for year: 2011.0
Processed 6111 documents for year 2011.0.
Processing documents for year: 2012.0
Processed 6461 documents for year 2012.0.
Processing documents for year: 2013.0
Processed 5534 documents for year 2013.0.
Processing documents for year: 2016.0
Processed 5377 documents for year 2016.0.
Processing documents for year: 2017.0


## 2. Get the coherence of each topic by year

**Reasoning**:
Calculate the semantic coherence for each topic for each year using the `CoherenceModel`.

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Initialize dictionary to store yearly topic coherence scores
yearly_topic_coherence = {}

# Check if yearly_processed_texts, model, and dictionary are loaded
if yearly_processed_texts and loaded_lda_model is not None and loaded_lda_model.id2word is not None:
    print("Calculating topic coherence for each year...")
    for year, processed_texts_for_year in yearly_processed_texts.items():
        print(f"Calculating topic coherence for year: {year}")
        if processed_texts_for_year:
            try:
                # Create a CoherenceModel instance
                coherence_model_lda_year = CoherenceModel(model=loaded_lda_model, texts=processed_texts_for_year, dictionary=loaded_lda_model.id2word, coherence='c_v')

                # Calculate the coherence scores for each topic for the current year's data
                topic_coherence_lda_year = coherence_model_lda_year.get_coherence_per_topic()

                # Store the calculated topic coherence scores
                yearly_topic_coherence[year] = topic_coherence_lda_year
                print(f"Calculated topic coherence for year {year}.")

            except Exception as e:
                print(f"An error occurred while calculating topic coherence for year {year}: {e}")
                yearly_topic_coherence[year] = None # Store None if calculation fails

        else:
            print(f"Skipping topic coherence calculation for year {year} due to empty processed texts.")
            yearly_topic_coherence[year] = None # Store None if skipping

    print("\nFinished calculating yearly topic coherence scores.")

    # Store and display the results in a DataFrame
    if yearly_topic_coherence:
        print("Storing and displaying yearly topic coherence scores...")
        yearly_coherence_series = pd.Series(yearly_topic_coherence)
        yearly_topic_coherence_df = pd.DataFrame(yearly_coherence_series.tolist(), index=yearly_coherence_series.index)
        yearly_topic_coherence_df.columns = [f'Topic {i}' for i in range(yearly_topic_coherence_df.shape[1])]
        yearly_topic_coherence_df = yearly_topic_coherence_df.sort_index()

        print("\nYearly Topic Coherence Scores:")
        display(yearly_topic_coherence_df)

        print("\nYearly topic coherence scores stored and displayed successfully.")
    else:
        print("No yearly topic coherence scores were calculated.")


else:
    print("'yearly_processed_texts' is not available or is empty, or model/dictionary missing. Cannot calculate yearly topic coherence scores.")

## 3. Get the exclusivity of each topic by year

**Reasoning**:
Calculate the semantic exclusivity for each topic for each year using a custom metric based on topic-word distributions.

In [ ]:
import numpy as np

# Initialize dictionary to store yearly topic exclusivity scores
yearly_topic_exclusivity = {}

# Check if yearly_processed_texts and loaded_lda_model are available
if yearly_processed_texts and loaded_lda_model is not None:
    print("Calculating topic exclusivity for each year...")
    # Get the topic-word distributions from the loaded model
    topic_word_distributions = loaded_lda_model.get_topics() # Shape (num_topics, vocab_size)
    num_topics = topic_word_distributions.shape[0]
    vocab_size = topic_word_distributions.shape[1]

    if num_topics > 1:
         # Iterate through the yearly_processed_texts dictionary
        for year, processed_texts_for_year in yearly_processed_texts.items():
            print(f"Calculating topic exclusivity for year: {year}")
            # Check if the processed texts for the current year are not empty
            if processed_texts_for_year:
                try:
                    # Calculate exclusivity for each topic for the current year
                    topic_exclusivity_scores_year = []
                    for topic_id in range(num_topics):
                        topic_word_dist = topic_word_distributions[topic_id, :] # p(w|z) for the current topic

                        # Calculate the probability of word w in any topic other than z: sum_{z' != z} p(w|z')
                        sum_p_w_other_topics = np.sum(np.delete(topic_word_distributions, topic_id, axis=0), axis=0)

                        # Using a simple metric based on p(w|z) * (p(w|z) - max_{z' != z} p(w|z'))
                        # Consider top words of the topic
                        top_word_indices = np.argsort(topic_word_dist)[::-1][:20] # Consider top 20 words

                        topic_exclusivity_score = 0
                        for word_idx in top_word_indices:
                            p_w_z = topic_word_dist[word_idx]
                            max_p_w_not_z = np.max(np.delete(topic_word_distributions, topic_id, axis=0)[:, word_idx])

                            word_exclusivity = p_w_z * (p_w_z - max_p_w_not_z)
                            topic_exclusivity_score += word_exclusivity

                        topic_exclusivity_scores_year.append(topic_exclusivity_score)

                    yearly_topic_exclusivity[year] = topic_exclusivity_scores_year
                    print(f"Calculated topic exclusivity for year {year}.")

                except Exception as e:
                    print(f"An error occurred while calculating topic exclusivity for year {year}: {e}")
                    yearly_topic_exclusivity[year] = None # Store None if calculation fails

            else:
                print(f"Skipping topic exclusivity calculation for year {year} due to empty processed texts.")
                yearly_topic_exclusivity[year] = None # Store None if skipping
        print("\nFinished calculating yearly topic exclusivity scores.")

        # Store and display the results in a DataFrame
        if yearly_topic_exclusivity:
            print("Storing and displaying yearly topic exclusivity scores...")
            yearly_exclusivity_series = pd.Series(yearly_topic_exclusivity)
            yearly_topic_exclusivity_df = pd.DataFrame(yearly_exclusivity_series.tolist(), index=yearly_exclusivity_series.index)
            yearly_topic_exclusivity_df.columns = [f'Topic {i}' for i in range(yearly_topic_exclusivity_df.shape[1])]
            yearly_topic_exclusivity_df = yearly_topic_exclusivity_df.sort_index()

            print("\nYearly Topic Exclusivity Scores:")
            display(yearly_topic_exclusivity_df)

            print("\nYearly topic exclusivity scores stored and displayed successfully.")
        else:
             print("No yearly topic exclusivity scores were calculated.")


    else:
        print("Skipping exclusivity calculation as there is only one topic.")


else:
    print("'yearly_processed_texts' is not available or is empty, or loaded_lda_model is missing. Cannot calculate yearly topic exclusivity scores.")

## 4. Visualize the coherences for each topic per year

**Reasoning**:
Create line plots for each topic showing its semantic coherence score over the years using the stored yearly topic coherence DataFrame.

In [ ]:
import matplotlib.pyplot as plt

# Check if yearly_topic_coherence_df is available and not empty
if 'yearly_topic_coherence_df' in locals() and not yearly_topic_coherence_df.empty:
    print("Generating line plots for each topic's coherence over time...")

    # Get the list of topic columns
    topic_columns = [col for col in yearly_topic_coherence_df.columns if col.startswith('Topic')]

    # Iterate through each topic column and create a plot
    for topic_col in topic_columns:
        plt.figure(figsize=(12, 6))
        # Plot the coherence score for the current topic against the years (index)
        plt.plot(yearly_topic_coherence_df.index, yearly_topic_coherence_df[topic_col], marker='o', linestyle='-')

        # Add title and axis labels
        plt.title(f'Semantic Coherence (c_v) for {topic_col} Over Time')
        plt.xlabel('Year')
        plt.ylabel('Coherence Score')

        # Add grid for better readability
        plt.grid(True)

        # Display the plot
        plt.show()

    print("\nFinished generating topic coherence plots.")

else:
    print("'yearly_topic_coherence_df' DataFrame not found or is empty. Cannot generate plots.")

## 5. Visualize the exclusivity for each topic per year

**Reasoning**:
Create line plots for each topic showing its semantic exclusivity score over the years using the stored yearly topic exclusivity DataFrame.

In [ ]:
import matplotlib.pyplot as plt

# Check if yearly_topic_exclusivity_df is available and not empty
if 'yearly_topic_exclusivity_df' in locals() and not yearly_topic_exclusivity_df.empty:
    print("Generating line plots for each topic's exclusivity over time...")

    # Get the list of topic columns
    topic_columns = [col for col in yearly_topic_exclusivity_df.columns if col.startswith('Topic')]

    # Iterate through each topic column and create a plot
    for topic_col in topic_columns:
        plt.figure(figsize=(12, 6))
        # Plot the exclusivity score for the current topic against the years (index)
        plt.plot(yearly_topic_exclusivity_df.index, yearly_topic_exclusivity_df[topic_col], marker='o', linestyle='-')

        # Add title and axis labels
        plt.title(f'Semantic Exclusivity for {topic_col} Over Time')
        plt.xlabel('Year')
        plt.ylabel('Exclusivity Score')

        # Add grid for better readability
        plt.grid(True)

        # Display the plot
        plt.show()

    print("\nFinished generating topic exclusivity plots.")

else:
    print("'yearly_topic_exclusivity_df' DataFrame not found or is empty. Cannot generate plots.")

## 6. Save the data and models in Google Drive

**Reasoning**:
Save the generated dataframes (yearly topic coherence and exclusivity) to Excel files and the dictionaries (yearly dataframes, yearly coherence, yearly exclusivity) to pickle files for later reuse.

In [ ]:
import os
import pickle
import pandas as pd

# Define the directory to save the files in your Google Drive
save_dir = '/content/drive/MyDrive/Colab Notebooks/alex/saved_analysis_data/30_topics_model'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# --- Save yearly_topic_coherence_df DataFrame to Excel ---
excel_coherence_path = os.path.join(save_dir, 'yearly_30_topics_coherence_scores.xlsx')
if 'yearly_topic_coherence_df' in locals() and yearly_topic_coherence_df is not None and not yearly_topic_coherence_df.empty:
    print(f"Saving yearly topic coherence scores to {excel_coherence_path}...")
    try:
        yearly_topic_coherence_df.to_excel(excel_coherence_path, index=True)
        print("Yearly topic coherence scores saved successfully to Excel.")
    except Exception as e:
        print(f"Error saving yearly topic coherence scores to Excel: {e}")
else:
    print("'yearly_topic_coherence_df' DataFrame not found or is empty. Skipping save.")


# --- Save yearly_topic_exclusivity_df DataFrame to Excel ---
excel_exclusivity_path = os.path.join(save_dir, 'yearly_30_topics_exclusivity_scores.xlsx')
if 'yearly_topic_exclusivity_df' in locals() and yearly_topic_exclusivity_df is not None and not yearly_topic_exclusivity_df.empty:
    print(f"Saving yearly topic exclusivity scores to {excel_exclusivity_path}...")
    try:
        yearly_topic_exclusivity_df.to_excel(excel_exclusivity_path, index=True)
        print("Yearly topic exclusivity scores saved successfully to Excel.")
    except Exception as e:
        print(f"Error saving yearly topic exclusivity scores to Excel: {e}")
else:
    print("'yearly_topic_exclusivity_df' DataFrame not found or is empty. Skipping save.")


# --- Save yearly_dataframes dictionary using pickle ---
yearly_dataframes_path = os.path.join(save_dir, 'yearly_30_topics_dataframes.pkl')
if 'yearly_dataframes' in locals() and yearly_dataframes:
    print(f"Saving yearly_dataframes to {yearly_dataframes_path}...")
    try:
        with open(yearly_dataframes_path, 'wb') as f:
            pickle.dump(yearly_dataframes, f)
        print("yearly_dataframes saved successfully.")
    except Exception as e:
        print(f"Error saving yearly_dataframes: {e}")
else:
    print("'yearly_dataframes' dictionary not found. Skipping save.")


# --- Save yearly_coherence_scores dictionary using pickle ---
# Note: This saves the overall yearly coherence scores, not topic-specific ones per year
yearly_coherence_scores_path = os.path.join(save_dir, 'yearly_30_topics_coherence_scores.pkl')
if 'yearly_coherence_scores' in locals() and yearly_coherence_scores:
    print(f"Saving yearly_coherence_scores to {yearly_coherence_scores_path}...")
    try:
        with open(yearly_coherence_scores_path, 'wb') as f:
            pickle.dump(yearly_coherence_scores, f)
        print("yearly_coherence_scores saved successfully.")
    except Exception as e:
        print(f"Error saving yearly_coherence_scores: {e}")
else:
    print("'yearly_coherence_scores' dictionary not found. Skipping save.")


# --- Save yearly_topic_coherence dictionary using pickle ---
# This saves the dictionary containing list of topic coherences per year
yearly_topic_coherence_dict_path = os.path.join(save_dir, 'yearly_30_topics_coherence_dict.pkl')
if 'yearly_topic_coherence' in locals() and yearly_topic_coherence:
    print(f"Saving yearly_topic_coherence dictionary to {yearly_topic_coherence_dict_path}...")
    try:
        with open(yearly_topic_coherence_dict_path, 'wb') as f:
            pickle.dump(yearly_topic_coherence, f)
        print("yearly_topic_coherence dictionary saved successfully.")
    except Exception as e:
        print(f"Error saving yearly_topic_coherence dictionary: {e}")
else:
    print("'yearly_topic_coherence' dictionary not found. Skipping save.")


# --- Save yearly_topic_exclusivity dictionary using pickle ---
# This saves the dictionary containing list of topic exclusivities per year
yearly_topic_exclusivity_dict_path = os.path.join(save_dir, 'yearly_30_topics_exclusivity_dict.pkl')
if 'yearly_topic_exclusivity' in locals() and yearly_topic_exclusivity:
    print(f"Saving yearly_topic_exclusivity dictionary to {yearly_topic_exclusivity_dict_path}...")
    try:
        with open(yearly_topic_exclusivity_dict_path, 'wb') as f:
            pickle.dump(yearly_topic_exclusivity, f)
        print("yearly_topic_exclusivity dictionary saved successfully.")
    except Exception as e:
        print(f"Error saving yearly_topic_exclusivity dictionary: {e}")
else:
    print("'yearly_topic_exclusivity' dictionary not found. Skipping save.")


print("\nSave process complete.")

## Summary

The notebook has been reorganized to follow the requested structure. The code cells for loading data, calculating and visualizing yearly topic coherence and exclusivity, and saving the results have been generated.

In [ ]:
%pip install reportlab

In [ ]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
import os
import matplotlib.pyplot as plt

# Define the directory to save temporary images and CSVs for the PDF
temp_dir = '/content/temp_pdf_assets'
os.makedirs(temp_dir, exist_ok=True)

# --- Save Plots as Images ---
image_paths = []
if 'yearly_topic_coherence_df' in locals() and not yearly_topic_coherence_df.empty:
    print("Saving coherence plots as images...")
    topic_columns = [col for col in yearly_topic_coherence_df.columns if col.startswith('Topic')]
    for topic_col in topic_columns:
        plt.figure(figsize=(12, 6)) # Increased figure size
        plt.plot(yearly_topic_coherence_df.index, yearly_topic_coherence_df[topic_col], marker='o', linestyle='-')
        plt.title(f'Semantic Coherence (c_v) for {topic_col} Over Time')
        plt.xlabel('Year')
        plt.ylabel('Coherence Score')
        plt.grid(True)
        img_path = os.path.join(temp_dir, f'{topic_col}_coherence.png')
        plt.savefig(img_path)
        image_paths.append(img_path)
        plt.close() # Close the plot to free up memory
    print("Finished saving coherence plots.")

if 'yearly_topic_exclusivity_df' in locals() and not yearly_topic_exclusivity_df.empty:
    print("Saving exclusivity plots as images...")
    topic_columns = [col for col in yearly_topic_exclusivity_df.columns if col.startswith('Topic')]
    for topic_col in topic_columns:
        plt.figure(figsize=(12, 6)) # Increased figure size
        plt.plot(yearly_topic_exclusivity_df.index, yearly_topic_exclusivity_df[topic_col], marker='o', linestyle='-')
        plt.title(f'Semantic Exclusivity for {topic_col} Over Time')
        plt.xlabel('Year')
        plt.ylabel('Exclusivity Score')
        plt.grid(True)
        img_path = os.path.join(temp_dir, f'{topic_col}_exclusivity.png')
        plt.savefig(img_path)
        image_paths.append(img_path)
        plt.close() # Close the plot to free up memory
    print("Finished saving exclusivity plots.")


# --- Save DataFrames as CSV for Table Inclusion ---
# Skipping CSV save as requested by the user
# coherence_csv_path = os.path.join(temp_dir, 'yearly_coherence_scores.csv')
# if 'yearly_topic_coherence_df' in locals() and yearly_topic_coherence_df is not None and not yearly_topic_coherence_df.empty:
#     yearly_topic_coherence_df.to_csv(coherence_csv_path, index=True)
#     print(f"Yearly topic coherence scores saved to {coherence_csv_path}")
# else:
#     coherence_csv_path = None
#     print("'yearly_topic_coherence_df' DataFrame not found or is empty. Skipping CSV save for coherence.")

# exclusivity_csv_path = os.path.join(temp_dir, 'yearly_exclusivity_scores.csv')
# if 'yearly_topic_exclusivity_df' in locals() and yearly_topic_exclusivity_df is not None and not yearly_topic_exclusivity_df.empty:
#     yearly_topic_exclusivity_df.to_csv(exclusivity_csv_path, index=True)
#     print(f"Yearly topic exclusivity scores saved to {exclusivity_csv_path}")
# else:
#     exclusivity_csv_path = None
#     print("'yearly_topic_exclusivity_df' DataFrame not found or is empty. Skipping CSV save for exclusivity.")


# --- Create PDF Report ---
pdf_path = '/content/drive/MyDrive/Colab Notebooks/alex/saved_analysis_data/30_topics_model/topic_analysis_report_charts_only_30_topics.pdf' # Changed filename
doc = SimpleDocTemplate(pdf_path, pagesize=letter) # Use portrait pagesize for charts only
story = []
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='TableText', fontSize=8)) # Keep TableText style just in case, though not used for tables now

# Add Title
story.append(Paragraph("Topic Analysis Report (20 Topics Model) - Charts Only", styles['Title']))
story.append(Spacer(1, 0.2*inch))

# Add Plots
story.append(Paragraph("Yearly Topic Coherence and Exclusivity Plots", styles['Heading2']))
story.append(Spacer(1, 0.1*inch))

for img_path in image_paths:
    if os.path.exists(img_path):
        img = Image(img_path, width=6*inch, height=3*inch) # Increased image size
        story.append(img)
        story.append(Spacer(1, 0.1*inch))

try:
    doc.build(story)
    print(f"\nPDF report generated successfully at {pdf_path}")
except Exception as e:
    print(f"Error generating PDF report: {e}")

# Optional: Clean up temporary files
# import shutil
# shutil.rmtree(temp_dir)
# print("Cleaned up temporary files.")

##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**